# 10 - Cloud Automation Samples / Amostras de Automação em Nuvem

This bilingual, hands-on notebook shows end-to-end snippets for: Google Cloud, AWS, and Azure. Topics include: model deployment, object storage (buckets), remote execution, CI/CD automation, and security tips.

Este notebook bilíngue e prático traz trechos de ponta a ponta para: Google Cloud, AWS e Azure. Tópicos incluem: deploy de modelos, armazenamento em buckets, execução remota, automação CI/CD e dicas de segurança.

## Prerequisites / Pré-requisitos
- Python 3.10+
- Credentials configured locally (gcloud, aws, az CLIs) or environment variables set.
- Appropriate IAM/permissions for the actions.

- Credenciais configuradas localmente (gcloud, aws, az) ou variáveis de ambiente.
- Permissões/IAM adequadas para as ações.

## 1) Google Cloud: Storage, Cloud Run deploy, and Batch remote job
## 1) Google Cloud: Storage, deploy no Cloud Run e job remoto no Batch

In [ ]:
# Install and imports (uncomment as needed)
# !pip install google-cloud-storage google-cloud-aiplatform google-cloud-run google-cloud-batch google-auth
from google.cloud import storage
import os, json, subprocess

PROJECT_ID = os.getenv('GOOGLE_CLOUD_PROJECT', 'your-project-id')
BUCKET = f'{PROJECT_ID}-ds-artifacts'
REGION = os.getenv('GOOGLE_CLOUD_REGION', 'us-central1')
SERVICE_NAME = 'demo-ml-api'
IMAGE = f'gcr.io/{PROJECT_ID}/{SERVICE_NAME}:latest'

# Create bucket if not exists / Cria bucket se não existir
client = storage.Client(project=PROJECT_ID)
buckets = [b.name for b in client.list_buckets()]
if BUCKET not in buckets:
    client.create_bucket(BUCKET, location=REGION)
    print('Created bucket:', BUCKET)
else:
    print('Bucket exists:', BUCKET)

# Upload a model artifact / Envia um artefato de modelo
blob_path = 'models/sklearn_demo/model.pkl'
blob = client.bucket(BUCKET).blob(blob_path)
blob.upload_from_filename('model.pkl')
print('Uploaded to gs://%s/%s' % (BUCKET, blob_path))

# Build & push image (requires gcloud auth) / Build e push de imagem
subprocess.run(['gcloud','builds','submit','--tag', IMAGE], check=True)

# Deploy to Cloud Run / Deploy no Cloud Run
subprocess.run(['gcloud','run','deploy', SERVICE_NAME, '--image', IMAGE, '--region', REGION, '--platform','managed','--allow-unauthenticated'], check=True)

# Submit remote job to Batch (example) / Submeter job remoto ao Batch (exemplo)
# Note: requires a job spec JSON file / requer um spec JSON
# subprocess.run(['gcloud','batch','jobs','submit','demo-batch','--location', REGION, '--config','batch_job.json'], check=True)


## 2) AWS: S3, SageMaker model deploy, and Batch job
## 2) AWS: S3, deploy no SageMaker e job no Batch

In [ ]:
# !pip install boto3 sagemaker
import boto3, os
session = boto3.session.Session()
region = session.region_name or os.getenv('AWS_REGION','us-east-1')
s3 = boto3.client('s3', region_name=region)
bucket = os.getenv('AWS_S3_BUCKET','your-ds-artifacts')
key = 'models/sklearn_demo/model.pkl'
s3.create_bucket(Bucket=bucket) if bucket not in [b['Name'] for b in s3.list_buckets()['Buckets']] else None
s3.upload_file('model.pkl', bucket, key)
print(f'Uploaded to s3://{bucket}/{key}')

# SageMaker deploy (high-level example)
import sagemaker
from sagemaker.sklearn.model import SKLearnModel
role = os.getenv('AWS_SAGEMAKER_ROLE','arn:aws:iam::123456789012:role/SageMakerExecutionRole')
model = SKLearnModel(model_data=f's3://{bucket}/{key}', role=role, entry_point='inference.py', framework_version='1.2-1')
predictor = model.deploy(instance_type='ml.m5.large', initial_instance_count=1)
print('Endpoint name:', predictor.endpoint_name)

# AWS Batch job submit (requires a Job Queue/Definition)
batch = boto3.client('batch', region_name=region)
# batch.submit_job(jobName='demo-batch', jobQueue='your-queue', jobDefinition='your-def', containerOverrides={'command':['python','task.py']})


## 3) Azure: Blob Storage, Azure ML Online Endpoint, and Batch
## 3) Azure: Blob Storage, Azure ML Endpoint Online e Batch

In [ ]:
# !pip install azure-storage-blob azure-ai-ml azure-identity
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient
from azure.ai.ml import MLClient
import os
cred = DefaultAzureCredential()
subscription_id = os.getenv('AZURE_SUBSCRIPTION_ID')
resource_group = os.getenv('AZURE_RESOURCE_GROUP')
workspace = os.getenv('AZURE_ML_WORKSPACE')
ml = MLClient(cred, subscription_id, resource_group, workspace)

# Blob Storage upload
conn_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
container = os.getenv('AZURE_BLOB_CONTAINER','ds-artifacts')
bsc = BlobServiceClient.from_connection_string(conn_str)
try:
    bsc.create_container(container)
except Exception:
    pass
bsc.get_container_client(container).upload_blob('models/sklearn_demo/model.pkl', data=open('model.pkl','rb'), overwrite=True)
print('Uploaded to Azure Blob')

# Azure ML: create online endpoint (YAML/specs required)
# ml.online_endpoints.begin_create_or_update(Endpoint(...))
# ml.online_deployments.begin_create_or_update(Deployment(...))
# ml.online_endpoints.invoke(endpoint_name, request_file='payload.json')


## 4) CI/CD automation examples (GitHub Actions)
## 4) Exemplos de automação CI/CD (GitHub Actions)

In [ ]:
# .github/workflows/deploy.yml (snippet)
deploy_yaml = '''
name: CI-CD-Cloud
on:
  push:
    branches: [ "main" ]
jobs:
  build-deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-python@v5
        with:
          python-version: '3.11'
      - name: Install deps
        run: pip install -r requirements.txt
      - name: Build image
        run: docker build -t ${{ secrets.GCP_IMAGE }} .
      - name: Auth GCP
        uses: google-github-actions/auth@v2
        with:
          credentials_json: ${{ secrets.GCP_SA_KEY }}
      - name: Push & Deploy Cloud Run
        run: |
          gcloud auth configure-docker gcr.io -q
          docker push ${{ secrets.GCP_IMAGE }}
          gcloud run deploy demo-ml-api --image ${{ secrets.GCP_IMAGE }} --region us-central1 --platform managed --allow-unauthenticated
'''
print(deploy_yaml)


## 5) Security tips / Dicas de segurança
- Never hardcode secrets. Use Secret Managers (GCP Secret Manager, AWS Secrets Manager, Azure Key Vault).
- Principle of least privilege for service accounts and roles.
- Enable artifact scanning and image vulnerability checks.
- Network policies and private endpoints where possible.
- Rotate credentials regularly and enforce MFA.

- Nunca codifique segredos no código. Use gerenciadores de segredos.
- Princípio do menor privilégio para contas/roles.
- Habilite varredura de artefatos e imagens.
- Políticas de rede e endpoints privados quando possível.
- Rode rotação de credenciais e MFA.

## 6) References / Referências
- Google Cloud: Cloud Run, Storage, Batch, Vertex AI: https://cloud.google.com/
- AWS: S3, SageMaker, Batch: https://aws.amazon.com/
- Azure: Blob Storage, Azure ML, Batch: https://azure.microsoft.com/
- Terraform for IaC: https://www.terraform.io/
- GitHub Actions: https://docs.github.com/actions
- Twelve-Factor App: https://12factor.net/